In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [5]:
os.path.join(data_dir, "train")

'hymenoptera_data\\train'

: 

In [3]:
data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)

['ants', 'bees']


In [4]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [12]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=100)




Epoch 0/99
----------
train Loss: 0.6324 Acc: 0.6598
val Loss: 0.4786 Acc: 0.7320

Epoch 1/99
----------
train Loss: 0.5248 Acc: 0.7459
val Loss: 0.3411 Acc: 0.9346

Epoch 2/99
----------
train Loss: 0.4554 Acc: 0.8074
val Loss: 0.2883 Acc: 0.9150

Epoch 3/99
----------
train Loss: 0.4349 Acc: 0.8033
val Loss: 0.2477 Acc: 0.9346

Epoch 4/99
----------
train Loss: 0.4189 Acc: 0.8320
val Loss: 0.2180 Acc: 0.9346

Epoch 5/99
----------
train Loss: 0.3882 Acc: 0.8402
val Loss: 0.2120 Acc: 0.9477

Epoch 6/99
----------
train Loss: 0.3887 Acc: 0.8197
val Loss: 0.1938 Acc: 0.9673

Epoch 7/99
----------
train Loss: 0.3315 Acc: 0.8730
val Loss: 0.1872 Acc: 0.9608

Epoch 8/99
----------
train Loss: 0.3456 Acc: 0.8770
val Loss: 0.2022 Acc: 0.9346

Epoch 9/99
----------
train Loss: 0.3302 Acc: 0.8893
val Loss: 0.1852 Acc: 0.9608

Epoch 10/99
----------
train Loss: 0.3208 Acc: 0.8770
val Loss: 0.1932 Acc: 0.9542

Epoch 11/99
----------
train Loss: 0.3593 Acc: 0.8361
val Loss: 0.1907 Acc: 0.9608

Ep

In [13]:

model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=100)

Epoch 0/99
----------
train Loss: 0.7759 Acc: 0.6025
val Loss: 0.1956 Acc: 0.9281

Epoch 1/99
----------
train Loss: 0.4377 Acc: 0.8033
val Loss: 0.2097 Acc: 0.9216

Epoch 2/99
----------
train Loss: 0.4266 Acc: 0.7992
val Loss: 0.6379 Acc: 0.7647

Epoch 3/99
----------
train Loss: 0.5026 Acc: 0.7910
val Loss: 0.1972 Acc: 0.9542

Epoch 4/99
----------
train Loss: 0.4587 Acc: 0.8033
val Loss: 0.1693 Acc: 0.9477

Epoch 5/99
----------
train Loss: 0.5070 Acc: 0.7787
val Loss: 0.2139 Acc: 0.9477

Epoch 6/99
----------
train Loss: 0.3688 Acc: 0.8279
val Loss: 0.2211 Acc: 0.9150

Epoch 7/99
----------
train Loss: 0.3623 Acc: 0.8566
val Loss: 0.1814 Acc: 0.9542

Epoch 8/99
----------
train Loss: 0.3687 Acc: 0.8566
val Loss: 0.1903 Acc: 0.9542

Epoch 9/99
----------
train Loss: 0.3316 Acc: 0.8689
val Loss: 0.1733 Acc: 0.9477

Epoch 10/99
----------
train Loss: 0.4423 Acc: 0.8279
val Loss: 0.1855 Acc: 0.9477

Epoch 11/99
----------
train Loss: 0.2905 Acc: 0.8934
val Loss: 0.1787 Acc: 0.9542

Ep